In [31]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [44]:
# load IMU data
trial_name = "tug_2"
quaternion_path = f"/Users/FranklinZhao/OpenSimProject/Dataset/vIMU/Experimental/{trial_name}/Rajapogal_FullBody_calibrated_orientations.sto"
velocity_path = f"/Users/FranklinZhao/OpenSimProject/Dataset/vIMU/Experimental/{trial_name}/Rajapogal_FullBody_calibrated_angular_velocity.sto"
acceleration_path = f"/Users/FranklinZhao/OpenSimProject/Dataset/vIMU/Experimental/{trial_name}/Rajapogal_FullBody_calibrated_linear_accelerations.sto"

# read IMU data into dataframes
quaternion_data = pd.read_csv(filepath_or_buffer=quaternion_path, sep='\t', header=4)
velocity_data = pd.read_csv(filepath_or_buffer=velocity_path, sep='\t', header=4)
acceleration_data = pd.read_csv(filepath_or_buffer=acceleration_path, sep='\t', header=4)

In [45]:
# load kinematics data
kinematics_path = "/Users/FranklinZhao/OpenSimProject/Dataset/Motion/Experimental/tug_02.mot"

# read kinematics data into dataframes
kinematics_data = pd.read_csv(filepath_or_buffer=kinematics_path, header=8, skipinitialspace=True, sep='\t')

In [46]:
# get random index for IMU sensor
def sampleSensor(sensorName):
    index = random.randint(0, 49)

    if index == 0:
        return sensorName
    else:
        return sensorName+str(index)

In [47]:
import numpy as np

def createIMUDataInstance(quaternion_data, velocity_data, acceleration_data):
    # Get indices for sensors to use in this data instance
    humerus_l_imu_tag = "humerus_l_imu"
    humerus_r_imu_tag = "humerus_r_imu"
    ulna_l_imu_tag = "ulna_l_imu"
    ulna_r_imu_tag = "ulna_r_imu"
    tags = [humerus_l_imu_tag, humerus_r_imu_tag, ulna_l_imu_tag, ulna_r_imu_tag]

    concatenated = []
    for tag in tags:
        raw_q = quaternion_data[tag].values
        raw_v = velocity_data[tag].values
        raw_a = acceleration_data[tag].values

        total = np.zeros([raw_q.shape[0]-1, 20])
        for i in range(0, raw_q.shape[0]-1):
            total[i, :4] = np.array(raw_q[i+1].split(',')).astype(float)
            total[i, 4:7] = np.array(raw_v[i+1].split(',')).astype(float)
            total[i, 7:10] = np.array(raw_a[i+1].split(',')).astype(float)
            total[i, 10:14] = np.array(raw_q[i].split(',')).astype(float)
            total[i, 14:17] = np.array(raw_v[i].split(',')).astype(float)
            total[i, 17:20] = np.array(raw_a[i].split(',')).astype(float)
        concatenated.append(total)

    concatenated_data = np.concatenate(concatenated, axis=1)
    return concatenated_data


In [48]:
def createDataInstance(masked = False):


    # get imu data at time index
    new_IMU_instance = createIMUDataInstance(quaternion_data, velocity_data, acceleration_data)
    # get all kinematics data at time index
    # new_kinematics_instance = kinematics_data.values[:, 1:]
    # [arm_flex_r(25), arm_add_r(26), arm_rot_r(27), elbow_flex_r(28), pro_sup_r(29)]
    # [arm_flex_l(32), arm_add_l(33), arm_rot_l(34), elbow_flex_l(35), pro_sup_l(36)]
    right_arm_kinematics = kinematics_data.values[:, 25:30]
    left_arm_kinematics = kinematics_data.values[:, 32:37]
    right_leg_kinematics = kinematics_data.values[:, 6:12]
    left_leg_kinematics = kinematics_data.values[:, 14:20]
    torso_kinematics = np.concatenate([kinematics_data.values[:, :6],  kinematics_data.values[:, 22:25]], axis=1)
    new_kinematics_instance = np.concatenate([left_arm_kinematics, 
                                              right_arm_kinematics, 
                                              left_leg_kinematics,
                                              right_leg_kinematics,
                                              torso_kinematics], axis = 1)

    if masked:
        new_kinematics_instance = np.zeros(new_kinematics_instance.shape)
    # concatenate and return
    new_data_instance = np.concatenate([new_IMU_instance, new_kinematics_instance[1:]], axis=1)
    plt.show()
    return new_data_instance

    

In [49]:
new_data_instance = createDataInstance(masked = True)
df = pd.DataFrame(new_data_instance)
file_path = '/Users/FranklinZhao/OpenSimProject/masked_data_instance2.csv'
df.to_csv(file_path, index=False, header=False)